In [1]:
import pytorch_lightning

from Reimplementation.data.DeepCDRDataModule import DeepCDRDataModule
from Reimplementation.models.DeepCDRLitModule import DeepCDRLitModule
from Reimplementation.util.config import *
%load_ext tensorboard

In [3]:
dpath = '../data'
drug_info_file = '%s/GDSC/1.Drug_listMon Jun 24 09_00_55 2019.csv' % dpath
cell_line_info_file = '%s/CCLE/Cell_lines_annotations_20181226.txt' % dpath
drug_feature_file = '%s/GDSC/drug_graph_feat' % dpath
genomic_mutation_file = '%s/CCLE/genomic_mutation_34673_demap_features.csv' % dpath
cancer_response_exp_file = '%s/CCLE/GDSC_IC50.csv' % dpath
gene_expression_file = '%s/CCLE/genomic_expression_561celllines_697genes_demap_features.csv' % dpath
methylation_file = '%s/CCLE/genomic_methylation_561celllines_808genes_demap_features.csv' % dpath
raw_files = NeededFiles(drug_info_file, cell_line_info_file, drug_feature_file, genomic_mutation_file,
                        cancer_response_exp_file, gene_expression_file, methylation_file)

GCN = ConvType.GCN
GAT = ConvType.GAT
tanh = ActivationFunc.Tanh
relu = ActivationFunc.ReLU
f_tanh = FuncActivationFunc.Tanh
f_relu = FuncActivationFunc.ReLU

drug_config = GNNConfig([75, 256, 256, 256], [GCN, GCN, GCN], [f_relu, f_relu, f_relu], False)
mutation_config = Conv1DConfig([1, 50, 30], [700, 5], [tanh, relu], [5, 10], False)
g_expr_config = FCConfig([256], [tanh], output=100, p_dropout=0.1, use_bn=False)
meth_config = FCConfig([256], [tanh], output=100, p_dropout=0.1, use_bn=False)
regressor_cnn_config = Conv1DConfig([1, 30, 10, 5], [150, 5, 5], [relu, relu, relu], [2, 3, 3], False)
regressor_fc_config = FCConfig([], [], output=1, p_dropout=0.1, use_bn=False)
config = DeepCDRConfig(drug_config, mutation_config, g_expr_config, meth_config, regressor_cnn_config, regressor_fc_config)

In [8]:
data = DeepCDRDataModule(raw_files, 100, batch_size=10)
model = DeepCDRLitModule(config, lr=0.000001)

/home/user/PycharmProjects/DeepCDR/venv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [5]:
data.get_debug_batch()
print(len(data.train_dataloader()))
print(len(data.val_dataloader()))
print(len(data.test_dataloader()))

6
2
2


In [9]:
import pytorch_lightning as pl
from  pytorch_lightning.tuner.tuning import Tuner
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

pl.seed_everything(0)
early_stopping = pl.callbacks.early_stopping.EarlyStopping(monitor="Validation loss", mode='min', stopping_threshold=0.005, patience=4)
logger=TensorBoardLogger(save_dir="log", name="DeepCDR_Logs", version="Only one final Linear Layer for regression with convolution")
trainer = pl.Trainer(logger=logger, callbacks=[early_stopping], log_every_n_steps=1)
debug_trainer = pl.Trainer(overfit_batches=1, logger=logger, max_epochs=40, log_every_n_steps=1)

Global seed set to 0
/home/user/PycharmProjects/DeepCDR/venv/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.


In [12]:
debug_trainer.fit(model, datamodule=data)


  | Name              | Type              | Params | In sizes              | Out sizes 
---------------------------------------------------------------------------------------------
0 | drug_gnn          | Sequential_23c6d8 | 151 K  | [[100, 75], [2, 101]] | [100, 256]
1 | mutation_cnn1d    | Sequential        | 2.1 M  | [1, 1, 34673]         | [1, 100]  
2 | g_expr_fcn        | Sequential        | 204 K  | [1, 697]              | [1, 100]  
3 | meth_fcn          | Sequential        | 232 K  | [1, 808]              | [1, 100]  
4 | regression_conv1d | Sequential        | 720 K  | [1, 1, 25900]         | [1, 100]  
5 | regression_fc     | Sequential        | 101    | [1, 100]              | [1, 1]    
---------------------------------------------------------------------------------------------
3.4 M     Trainable params
0         Non-trainable params
3.4 M     Total params
13.554    Total estimated model params size (MB)


In [ ]:
# Loading TensorBoard
%reload_ext tensorboard
%tensorboard --logdir log/